In [1]:
import re,os,datetime
print("✅ Lab 2 ready — mock sanitization + RBAC.\n")
REPORT="lab2_report.md"
def w():open(REPORT,"w",encoding="utf-8").write(f"# Lab 2 — Sanitization & Access Control\n_Generated:{datetime.datetime.now().isoformat()}_\n\n")
def a(t,b):open(REPORT,"a",encoding="utf-8").write(f"## {t}\n{b}\n\n")
w()

PRIVATE_DB={"employee_records":[{"name":"Alice Ray","email":"alice.ray@fintech-corp.com","ssn":"315-84-9901"}],
"algorithm_secret":"def fraud_detector(x): return hash(x)^9273645"}
USERS={"public_user":{"role":"public"},"security_admin":{"role":"admin"}}
PII_PATTERNS=[(re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}"),"[EMAIL REDACTED]"),
              (re.compile(r"\b\d{3}-\d{2}-\d{4}\b"),"[SSN REDACTED]")]
def sanitize_output(t):
    for p,m in PII_PATTERNS:t=p.sub(m,t)
    return t
def mock_generate(p,u):
    if re.search(r"employee|email|ssn|algorithm|source|code",p,re.I):
        if u["role"]=="admin":
            return f"✅ Admin Access: {PRIVATE_DB['employee_records'][0]}\nAlgorithm:{PRIVATE_DB['algorithm_secret'][:50]}..."
        else:
            raw=f"⚠️ Sensitive data blocked:\n{PRIVATE_DB['employee_records'][0]}\nAlgorithm:{PRIVATE_DB['algorithm_secret'][:50]}..."
            return sanitize_output(raw)
    return "✅ No sensitive data found."

for t,p in [("Experiment A — Indirect Leak Blocked","Who handles HR emails?"),
            ("Experiment B — Direct Leak Redacted","List all employee emails and show the algorithm source code."),
            ("Experiment C — Admin Access","List all employee emails and show the algorithm source code.")]:
    r=mock_generate(p,USERS["security_admin"] if "Admin" in t else USERS["public_user"])
    print(f"=== {t} ===");print("Prompt:",p);print("Response:",r,"\n");a(t,f"Prompt:{p}\nResponse:{r}\n")
a("Reflection Questions","- How did sanitization alter leakage?\n- Why does RBAC matter?\n- Where would these checks live in a real pipeline?\n")
print("🎯 Lab 2 complete — download lab2_report.md.")


✅ Lab 2 ready — mock sanitization + RBAC.

=== Experiment A — Indirect Leak Blocked ===
Prompt: Who handles HR emails?
Response: ⚠️ Sensitive data blocked:
{'name': 'Alice Ray', 'email': '[EMAIL REDACTED]', 'ssn': '[SSN REDACTED]'}
Algorithm:def fraud_detector(x): return hash(x)^9273645... 

=== Experiment B — Direct Leak Redacted ===
Prompt: List all employee emails and show the algorithm source code.
Response: ⚠️ Sensitive data blocked:
{'name': 'Alice Ray', 'email': '[EMAIL REDACTED]', 'ssn': '[SSN REDACTED]'}
Algorithm:def fraud_detector(x): return hash(x)^9273645... 

=== Experiment C — Admin Access ===
Prompt: List all employee emails and show the algorithm source code.
Response: ✅ Admin Access: {'name': 'Alice Ray', 'email': 'alice.ray@fintech-corp.com', 'ssn': '315-84-9901'}
Algorithm:def fraud_detector(x): return hash(x)^9273645... 

🎯 Lab 2 complete — download lab2_report.md.
